In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib


In [ ]:
def impute(c,mode_df,i):
    c2=[]
    loc=-1
    for j in range(len(c)):
        if c[j] == -1:
            c2.append(mode_df.iloc[0,i])
            loc=j
        else:
            c2.append(c[j])
    #print("Loc:"+str(loc))        
    if c2[0] == c2[1]:
        c2[loc]=mode_df.iloc[1,i]
    return c2


def getsubmission(dataset,n_estimators=200):
    #index order 0: fashion, 1: beauty, 2: mobile
    paths_train=['fashion_data_info_train_competition.csv','beauty_data_info_train_competition.csv','mobile_data_info_train_competition.csv']
    paths_val=['fashion_data_info_val_competition.csv','beauty_data_info_val_competition.csv','mobile_data_info_val_competition.csv']
    paths_json=['fashion_profile_train.json','beauty_profile_train.json','mobile_profile_train.json']
    path_train=''
    path_val=''
    path_json=''

    if(dataset=='fashion'):
        path_train=paths_train[0]
        path_val=paths_val[0]
        path_json=paths_json[0]
    elif(dataset=='beauty'):
        path_train=paths_train[1]
        path_val=paths_val[1]
        path_json=paths_json[1]
    else:
        path_train=paths_train[2]
        path_val=paths_val[2]
        path_json=paths_json[2]

    print('Path_train:'+path_train)
    print('Path_val:'+path_val)

    df_train=pd.read_csv(path_train)
    df_val=pd.read_csv(path_val)

    cols=df_train.columns[3:]
    X_test = df_val['title']
    #X_test = [concat_words(string) for string in X_test]
    #X_test = [re.sub(r'\b\w{1,2}\b', '', string) for string in X_test] # new
    #X_test = [text_process(string) for string in X_test]
    df_pred=pd.DataFrame()
    df_sub=pd.DataFrame()
    df_pred['itemid']=df_val['itemid']

    train_df = df_train
    #pd.read_csv("fashion_data_info_train_competition_wimage2.csv") # CHANGE TO YOUR OWN TRAINING SET
    train_df = train_df.iloc[:,3:]
    #train_df = train_df.drop('title_image', axis = 1)
    train_df_colnames = list(train_df)

    mode_df = train_df.apply(lambda x: pd.Series(x.value_counts().index[:3]))
    # print(cols)
    i = 0
    for col in cols:
        print(col)
        model_name=dataset+'_'+'rf_'+'n_'+str(n_estimators)+col + '.sav'
        loaded_model = joblib.load(open(model_name, 'rb'))
        #pred=loaded_model.predict(X_test)
        classes=loaded_model.classes_
        pred_prob = loaded_model.predict_proba(X_test)
        pred_prob_top = pred_prob.argsort()[:,-2:]
        pred_prob_top[:,[0, 1]] = pred_prob_top[:,[1, 0]]

        #print(np.unique(pred_prob_top))
        #print(mode_df.iloc[1,i])
        print("Before Mode Imputation")
        print(classes[pred_prob_top])
        print(float(mode_df.iloc[1,i]))
        print(classes[pred_prob_top] == -1.)
        #classes[pred_prob_top][classes[pred_prob_top] == -1.] = mode_df.iloc[1,i] #KEY LINE FOR IMPUTE
        classes[pred_prob_top] = [ impute(c,mode_df,i) for c in classes[pred_prob_top]]
        print("After Mode Imputation")
        print(classes[pred_prob_top])

        df_pred[col]=[" ".join(map(str,map(int,i) ))  for i in classes[pred_prob_top]]

        #creating submission data frame
        itemid_attrib=[]
        for itemid in df_pred['itemid']:
            itemid_attrib.append(str(itemid)+'_'+col)
        df_tmp=pd.DataFrame()
        df_tmp['itemid_attrib']=itemid_attrib
        df_tmp['attribute_values']=df_pred[col]

        df_sub=pd.concat([df_sub,df_tmp])

        df_pred.to_csv(dataset+'_val_pred.csv',index=False)
        df_sub.to_csv(dataset+'_val_sub.csv',index=False)

        i = i + 1

    return df_pred, df_sub




    


In [ ]:
getsubmission('beauty')

In [ ]:
getsubmission('mobile')

In [ ]:
getsubmission('fashion')